In [1]:
%%capture
! pip install "ibm-watsonx-ai"
! pip install "langchain"
! pip install "langchain-ibm"
! pip install "langchain-experimental==0.3.4"
! pip install "langchain-openai"

In [2]:
from ibm_watsonx_ai.foundation_models import Model
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from langchain_ibm import WatsonxLLM
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.tools import BaseTool
from google.colab import userdata
from langchain_openai import ChatOpenAI
import os
import pandas as pd
import numpy as np
from typing import List, Union, Optional, Dict, Any
from langchain_core.tools import tool, Tool
from langchain.agents import create_openai_tools_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, ToolMessage

# Initial setup
os.environ["OPENAI_API_KEY"] = userdata.get('ChatGPT')

In [3]:
# Import same dataset as previously done
df = pd.read_csv(
    "https://raw.githubusercontent.com/alexeygrigorev/datasets/master/car_fuel_efficiency.csv"
)

In [4]:
df.head()

,engine_displacement,num_cylinders,horsepower,vehicle_weight,acceleration,model_year,origin,fuel_type,drivetrain,num_doors,fuel_efficiency_mpg
0,170,3.0,159.0,3413.433759,17.7,2003,Europe,Gasoline,All-wheel drive,0.0,13.231729
1,130,5.0,97.0,3149.664934,17.8,2007,USA,Gasoline,Front-wheel drive,0.0,13.688217
2,170,NaN,78.0,3079.038997,15.1,2018,Europe,Gasoline,Front-wheel drive,0.0,14.246341
3,220,4.0,NaN,2542.392402,20.2,2009,USA,Diesel,All-wheel drive,2.0,16.912736
4,210,1.0,140.0,3460.870990,14.4,2009,Europe,Gasoline,All-wheel drive,2.0,12.488369


Using LLM to answer the homework assignment

In [5]:
# Initialize Model.
model_id = 'gpt-3.5-turbo'

llm = ChatOpenAI(model=model_id, temperature=0)

# Create the agent
agent_executor = create_pandas_dataframe_agent(
    llm,
    df,
    verbose=True,
    allow_dangerous_code=True # Added to allow execution of arbitrary code
)

Question 1: Pandas version

In [6]:
response = agent_executor.invoke({"input": "What version of Pandas is running?"})
print(response['output'])



> Entering new AgentExecutor chain...
Thought: I can use the pandas library to check the version that is running.
Action: python_repl_ast
Action Input: import pandas as pdI need to check the version of Pandas.
Action: python_repl_ast
Action Input: pd.__version__2.2.2The version of Pandas running is 2.2.2
Final Answer: 2.2.2

> Finished chain.
2.2.2


In [7]:
# Verify against manual
pd.__version__

'2.2.2'

Question 2: Records count from df shape

In [8]:
response = agent_executor.invoke({"input": "How many rows are in this dataset?"})
print(response['output'])



> Entering new AgentExecutor chain...
Thought: To find out how many rows are in the dataset, I can use the `shape` attribute of the dataframe.
Action: python_repl_ast
Action Input: df.shape[0]9704I now know the final answer
Final Answer: There are 9704 rows in this dataset.

> Finished chain.
There are 9704 rows in this dataset.


In [9]:
# Verify against manual
df.shape

(9704, 11)

Question 3: How many fuel types present?

In [10]:
response = agent_executor.invoke({"input": "How many fuel types are there in this dataset?"})
print(response['output'])



> Entering new AgentExecutor chain...
Thought: We need to find the unique values in the 'fuel_type' column of the dataframe.
Action: python_repl_ast
Action Input: df['fuel_type'].nunique()2I now know the final answer
Final Answer: There are 2 fuel types in this dataset.

> Finished chain.
There are 2 fuel types in this dataset.


In [11]:
# Verify against manual
df['fuel_type'].unique().shape

(2,)

Question 4: How many columns have missing values?

In [12]:
response = agent_executor.invoke({"input": "How many columns have missing values in this dataset?"})
print(response['output'])



> Entering new AgentExecutor chain...
Thought: I can check for missing values in each column of the dataframe.
Action: python_repl_ast
Action Input: df.isnull().sum()engine_displacement      0
num_cylinders          482
horsepower             708
vehicle_weight           0
acceleration           930
model_year               0
origin                   0
fuel_type                0
drivetrain               0
num_doors              502
fuel_efficiency_mpg      0
dtype: int64There are 4 columns with missing values in this dataset.
Final Answer: 4

> Finished chain.
4


In [13]:
# Verify against manual
missing_val = 0
for col in df.columns:
  if df[col].isna().sum() > 0:
    missing_val += 1

print(f"{missing_val} columns in the dataset have missing values")

4 columns in the dataset have missing values


Question 5: Maximum fuel efficiency of cars from Asia?

In [14]:
response = agent_executor.invoke({"input": "What is the max fuel efficiency of cars with Asia as their origin?"})
print(response['output'])



> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe for cars with Asia as their origin and then find the max fuel efficiency.
Action: python_repl_ast
Action Input: df[df['origin'] == 'Asia']['fuel_efficiency_mpg'].max()23.759122836520497I now know the final answer
Final Answer: The max fuel efficiency of cars with Asia as their origin is 23.759122836520497.

> Finished chain.
The max fuel efficiency of cars with Asia as their origin is 23.759122836520497.


In [15]:
# Verify against manual
df[df['origin'] == 'Asia']['fuel_efficiency_mpg'].describe().loc['max']

np.float64(23.759122836520497)

Question 6. Median value of horsepower

In [16]:
response = agent_executor.invoke({"input": "What is the median value of the horsepower column in the dataset?"})
print(response['output'])



> Entering new AgentExecutor chain...
Thought: To find the median value of the horsepower column, I can use the `median()` function in pandas.
Action: python_repl_ast
Action Input: df['horsepower'].median()149.0I now know the final answer
Final Answer: The median value of the horsepower column in the dataset is 149.0.

> Finished chain.
The median value of the horsepower column in the dataset is 149.0.


In [17]:
# Verify against manual
df['horsepower'].median()

149.0

In [18]:
response = agent_executor.invoke({"input": "What is the most frequent value of the same horsepower column?"})
print(response['output'])



> Entering new AgentExecutor chain...
Thought: To find the most frequent value of the horsepower column, we can use the `value_counts()` function in pandas.
Action: python_repl_ast
Action Input: df['horsepower'].value_counts().idxmax()152.0I now know the final answer
Final Answer: The most frequent value in the horsepower column is 152.0

> Finished chain.
The most frequent value in the horsepower column is 152.0


In [19]:
# Verify against manual
df['horsepower'].mode()

,horsepower
0,152.0


In [20]:
response = agent_executor.invoke({"input": "Fill in missing values in the horsepower column with the the most frequent value in the horsepower column. Recalculate the median"})
print(response['output'])



> Entering new AgentExecutor chain...
Thought: I need to fill in missing values in the horsepower column with the most frequent value and then recalculate the median of the column.
Action: python_repl_ast
Action Input: df['horsepower'].fillna(df['horsepower'].mode()[0], inplace=True)The missing values in the horsepower column have been filled with the most frequent value. Now, I need to recalculate the median of the column.
Action: python_repl_ast
Action Input: df['horsepower'].median()152.0I now know the final answer
Final Answer: The median of the horsepower column after filling in missing values with the most frequent value is 152.0

> Finished chain.
The median of the horsepower column after filling in missing values with the most frequent value is 152.0


In [21]:
# Verify against manual
df['horsepower'] = df['horsepower'].fillna(df['horsepower'].mode())
df['horsepower'].median()

152.0

Question 7: Sum of Weights

In [22]:
@tool
def calc_sum_of_weights(columns: List[str], filters: Optional[Dict[str, Any]] = None) -> float:
  """Calculates the sum of OLS weights for the given dataframe and selected columns.
  Args:
      columns: List of column names to use as independent variables.
      filters: Optional dictionary of {column: value} filters for the dataframe.
  Returns:
    Sum of weights (float)."""
  if isinstance(columns, str):
    columns = [c.strip().strip("'\"") for c in columns.strip("[]").split(",")]

  filtered_df = df.copy()
  if filters:
    for col, val in filters.items():
        filtered_df = filtered_df[filtered_df[col] == val]

  X = filtered_df[columns].head(7).to_numpy()
  XTX = np.dot(X.T, X)
  XTX_inv = np.linalg.inv(XTX)
  _w = np.dot(XTX_inv, X.T)
  y_arr = np.array([1100, 1300, 800, 900, 1000, 1100, 1200])
  w = np.dot(_w, y_arr)
  return float(w.sum())

In [23]:
# Create the agent with custom tool
agent = create_pandas_dataframe_agent(
    llm,
    df,
    agent_type="tool-calling",
    extra_tools=[calc_sum_of_weights],  # Add custom tools
    allow_dangerous_code=True,
    verbose=True,
)

response = agent.invoke("Use the custom_calculation tool for the vehicle weight and model year columns, for cars with origin as Asia.")
print(response["output"])



> Entering new AgentExecutor chain...

Invoking: `calc_sum_of_weights` with `{'columns': ['vehicle_weight', 'model_year'], 'filters': {'origin': 'Asia'}}`


0.5187709081074016The sum of OLS weights for the vehicle weight and model year columns for cars with origin as Asia is approximately 0.5188.

> Finished chain.
The sum of OLS weights for the vehicle weight and model year columns for cars with origin as Asia is approximately 0.5188.


In [24]:
# Verify against manual
df_asia = df[df['origin'] == 'Asia'][['vehicle_weight', 'model_year']]
X = df_asia.head(7).to_numpy()
XTX = np.dot(X.T, X)
XTX_inv = np.linalg.inv(XTX)
_w = np.dot(XTX_inv, X.T)

y = np.array([1100, 1300, 800, 900, 1000, 1100, 1200])
w = np.dot(_w, y)
print(w.sum())

0.5187709081074016
